In [2]:
import os
import numpy as np

import shutil
from PIL import Image
import cv2
import cv2 as cv
import random

import urllib
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt

In [3]:
ROOT_DIR = os.getcwd()
ROOT_DIR

'C:\\Users\\HK Joshi\\Documents\\Github\\major_project\\biocube'

In [3]:
data_path = os.path.join(ROOT_DIR, 'datasets', 'VERA-fingervein')
cropped_data_path = os.path.join(ROOT_DIR, 'datasets',  'cropped-VERA-fingervein')
new_data_path = os.path.join(ROOT_DIR, 'datasets',  'LR-VERA-fingervein')
fingervein_feature_path = os.path.join(ROOT_DIR, 'datasets',  'LR-VERA-fingervein-max-curvature-images')

## 1. Crop images 50 pixels from top, left and bottom to remove bachground info

In [5]:
os.chdir(ROOT_DIR)
os.chdir(data_path)

for folder in os.listdir():
    if folder == 'METADATA.ods':
            continue
    os.chdir(folder)
    for file_name in os.listdir():
        img = np.array(Image.open(file_name).convert("RGB"))
        new_img = Image.fromarray(img[50:-51, 50: ,:]) #[h, w, c]
#         plt.imshow(img)
#         plt.show()
#         plt.imshow(new_img)
#         plt.show()
        
        new_file_path = os.path.join(cropped_data_path, folder)
        if os.path.isdir(new_file_path) == False:
            os.makedirs(new_file_path)
        dest = os.path.join(new_file_path)
        new_img.save(os.path.join(dest, f"{file_name.split('.')[0]}.{file_name.split('.')[1]}"))
    os.chdir('..')

# 2. Separate Left and Right fingervein dataset into different class

In [6]:
os.chdir(ROOT_DIR)
os.chdir(cropped_data_path)

for folder in os.listdir():
    # print(folder)
    if folder == 'METADATA.ods':
            continue
    os.chdir(folder)
    for file_name in os.listdir():
        if file_name.split('_')[1] == 'L':
            new_path = os.path.join(new_data_path, folder+'-L')
            if os.path.isdir(new_path) == False:
                os.makedirs(new_path)
            dest = os.path.join(new_path)
            shutil.copy2(file_name, dest)
        else:
            new_path = os.path.join(new_data_path, folder+'-R')
            if os.path.isdir(new_path) == False:
                os.makedirs(new_path)
            dest = os.path.join(new_path)
            shutil.copy2(file_name, dest)

    os.chdir('..')
os.chdir(ROOT_DIR)

# 3. Read images from a folder and rotate it for data augmentation

There are 2 images in a folder. Read the two images, take a random number between [-7, 7] that represents angle in degree. Rotate each image by given angle and save it. The new image name will be: "\<old_img_name\>rot\<degree\>.\<image_format\>"

In [7]:
degree_to_turn = 7

def gen_rot_img(img_lst):
    [path1, img1], [path2, img2] = img_lst
#     print(path1, img1, path2, img2)
    while(True): #Find the two values of angle such that none of them is zero
        deg1 = random.randint((-1 * degree_to_turn), degree_to_turn)
        deg2 = random.randint((-1 * degree_to_turn), degree_to_turn)
        # Return two degree angles who are different and not 0
        if (deg1 != 0 or deg2 != 0) and (deg1 != deg2):
            break
            
    rot_img11, rot_img12 = img1.rotate(deg1), img1.rotate(deg2)
    rot_img21, rot_img22 = img2.rotate(deg1), img2.rotate(deg2)
    
    rot_img11.save(f"{path1.split('.')[0]}rot{deg1}.{path1.split('.')[1]}"), rot_img12.save(f"{path1.split('.')[0]}rot{deg2}.{path1.split('.')[1]}")
    rot_img21.save(f"{path2.split('.')[0]}rot{deg1}.{path2.split('.')[1]}"), rot_img22.save(f"{path2.split('.')[0]}rot{deg2}.{path2.split('.')[1]}")
#     plt.subplot(141)
#     plt.imshow(rot_img11)
#     plt.subplot(142)
#     plt.imshow(rot_img12)
#     plt.subplot(143)
#     plt.imshow(rot_img21)
#     plt.subplot(144)
#     plt.imshow(rot_img22)
    

In [8]:
os.chdir(ROOT_DIR)
os.chdir(new_data_path)

for folder in os.listdir():
    files_in_folder = []
    os.chdir(folder)
    
    # Read the original two images in the folder
    for file in os.listdir():
        img = Image.open(file).convert("RGB")
        files_in_folder.append([file, img])
    
    # Create augmented images for both the images
    gen_rot_img(files_in_folder)
    
    os.chdir('..')
os.chdir(new_data_path)

# 4. Augmentation for Conditional GAN

In [23]:
os.chdir(ROOT_DIR)

# Define the path to the folder containing your JAB code images
# jabcode_folder = r'datasets\jabcode_exchange_replace'
jabcode_folder = r'datasets\jabcode_exchange_XOR'

# Define the output folder for augmented data
# output_folder = r'datasets\jabcode_exchange_replace_augmented'
output_folder = r'datasets\jabcode_exchange_XOR_augmented'

# Ensure the output folder exists, create it if it doesn't
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to create augmented images
def augment_image(image, output_folder, user_id, num_augmentations=10):
    for i in range(1, num_augmentations+1):
        augmented_image = image.copy()
        
        # Apply data augmentation here
        # Example: Rotate the image by a random angle
        angle = np.random.uniform(-10, 10)  # Adjust the range as needed
        rows, cols, _ = augmented_image.shape
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        augmented_image = cv2.warpAffine(augmented_image, M, (cols, rows))
        
        # Save the augmented image to the user's folder
        output_filename = f'jabcode_user_{user_id}_augmented_{i}.png'
        output_path = os.path.join(output_folder, output_filename)
        cv2.imwrite(output_path, augmented_image)

# Loop through the JAB code files and apply data augmentation
for foldername in os.listdir(jabcode_folder):
    # print(foldername)
    for filename in os.listdir(os.path.join(jabcode_folder, foldername)):
        # print(filename)
        user_id = int(foldername.split('_')[-1])
        if filename.endswith('.png'):
            user_folder = os.path.join(output_folder, foldername)
            if not os.path.exists(user_folder):
                os.makedirs(user_folder)

            image_path = os.path.join(jabcode_folder, foldername, filename)
            image = cv2.imread(image_path)
            # plt.imshow(image)
            # plt.show()
            
            # Apply data augmentation for each image
            augment_image(image, user_folder, user_id, num_augmentations=20)
#         break
#     break

## 5. Script to generate jabcode and return an image

In [25]:
def download_jabcode(bhash_no, file_name, close_driver_flag=False):
''' Saves the file in the given directory
    Input:
        bhash_no: Biohash number for which we need the jabcode
        file_name: Filename of the jabcode file (mention the extension as well). Mention the full path to store in a specific directory.
        close_driver_flag: Flag to indicate last file, to close the driver 
'''
    chromedriver_path = r'C:\Users\HK Joshi\Documents\Github\major_project\biocube\chrome-win64'

    # Suppress Permission error
    os.chmod(chromedriver_path, 755)

    # Desired URL
    url = "https://jabcode.org/create"

    # create a new Chrome session
    driver = webdriver.Chrome()
    driver.implicitly_wait(30)
    driver.get(url)
    
    # Find the textbox where we insert Biohash 
    text_area = driver.find_element(By.XPATH, '//*[@id="gatsby-focus-wrapper"]/div/div/div/div/div/div[2]/div[2]/div[1]/div[1]/textarea')
    text_area.send_keys(bhash_no)
    driver.implicitly_wait(5)

    # Find the submit button
    submit_biohash = driver.find_element(By.XPATH, '//*[@id="gatsby-focus-wrapper"]/div/div/div/div/div/div[2]/div[2]/div[1]/div[2]/div/button')
    submit_biohash.click()
    driver.implicitly_wait(5)
    
    img_url = driver.find_element(By.XPATH, '//*[@alt="jab code"]').get_attribute("src")
    urllib.request.urlretrieve(img_url, file_name)
    
    if close_driver_flag == True:
        driver.close()

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os

In [7]:
# Define the path to the image you want to augment
input_image_path = os.path.join(os.getcwd(), 'datasets', 'labelled_jabcode')
output_directory = os.path.join(os.getcwd(), 'datasets', 'augmented_labelled_jabcode')
    
for jabcode in os.listdir(input_image_path):
    for image_name in os.listdir(os.path.join(input_image_path, jabcode)):
        
        img_path = os.path.join(input_image_path, jabcode, image_name)
        save_dir = os.path.join(output_directory, jabcode)
        # Create a directory to store augmented images
        os.makedirs(save_dir, exist_ok=True)

        # Load the image
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # Configure the data generator for augmentation
        datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        # Generate augmented images
        num_samples = 100  # You can adjust this based on your needs

        for i, batch in enumerate(datagen.flow(x, batch_size=1, save_to_dir=save_dir, save_prefix='aug', save_format='jpeg')):
            if i >= num_samples:
                break  # Break the loop after generating the desired number of samples


## 6. Duplicate images in directory

In [9]:
dataset_dir = os.path.join(os.getcwd(), 'datasets', 'jabcode_exchange_replace')

for folder_name in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder_name)
    # print(folder_path)
    # Check if the path is a directory
    if os.path.isdir(folder_path):
        # Define the source and destination paths for copying
        source_path = os.path.join(folder_path, '1.png')
        destination_path = os.path.join(folder_path, '2.png')

        # Copy the image
        shutil.copyfile(source_path, destination_path)